
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 基本分類モデル

:label: `sec_classification`

回帰の場合、最初からの実装とフレームワーク機能を使用した簡潔な実装が非常に似ていることに気付いたかもしれません。分類についても同様です。本書の多くのモデルは分類を扱っているため、特にこの設定をサポートする機能を追加する価値があります。このセクションでは、将来のコードを簡素化するための分類モデルの基本クラスを提供します。


In [1]:
import torch
from d2l import torch as d2l


## `Classifier`クラス



以下に`Classifier`クラスを定義します。 `validation_step`では、検証バッチの損失値と分類精度の両方を報告します。 `num_val_batches`バッチごとに更新を描画します。これには、検証データ全体で平均化された損失と精度を生成できるという利点があります。最後のバッチに含まれるサンプルの数が少ない場合、これらの平均値は正確には正確ではありませんが、コードを単純にするためにこの小さな違いは無視します。


In [2]:
class Classifier(d2l.Module):  #@save
    """The base class of classification models."""
    def validation_step(self, batch):
        Y_hat = self(*batch[:-1])
        self.plot('loss', self.loss(Y_hat, batch[-1]), train=False)
        self.plot('acc', self.accuracy(Y_hat, batch[-1]), train=False)


デフォルトでは、線形回帰のコンテキストで行ったのと同じように、ミニバッチで動作する確率的勾配降下オプティマイザーを使用します。


In [3]:
@d2l.add_to_class(d2l.Module)  #@save
def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=self.lr)


## 正確さ

予測確率分布`y_hat`が与えられた場合、ハード予測を出力する必要がある場合は、通常、予測確率が最も高いクラスを選択します。実際、多くのアプリケーションでは選択が必要です。たとえば、Gmail では電子メールを「プライマリ」、「ソーシャル」、「アップデート」、「フォーラム」、または「スパム」に分類する必要があります。内部的に確率を推定することもありますが、最終的にはクラスの中から 1 つを選択する必要があります。

予測がラベル クラス`y`と一致する場合、予測は正しいことになります。分類精度は、すべての予測のうち正しい部分の割合です。精度を直接最適化することは難しい場合がありますが (微分可能ではありません)、多くの場合、精度が最も重視されるのはパフォーマンスの尺度です。多くの場合、これはベンチマークに関連する*量*です。そのため、分類器をトレーニングする際には、ほぼ常にそれを報告します。

精度は次のように計算されます。まず、 `y_hat`が行列の場合、2 番目の次元に各クラスの予測スコアが格納されると仮定します。 `argmax`を使用して、各行の最大エントリのインデックスによって予測クラスを取得します。次に、 [**予測されたクラスをグラウンドトゥルース`y`と要素ごとに比較します。** ] 等価演算子`==`データ型に依存するため、 `y_hat`のデータ型を`y`のデータ型と一致するように変換します。結果は、0 (偽) と 1 (真) のエントリを含むテンソルです。合計を計算すると、正しい予測の数が得られます。


In [4]:
@d2l.add_to_class(Classifier)  #@save
def accuracy(self, Y_hat, Y, averaged=True):
    """Compute the number of correct predictions."""
    Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
    preds = Y_hat.argmax(axis=1).type(Y.dtype)
    compare = (preds == Y.reshape(-1)).type(torch.float32)
    return compare.mean() if averaged else compare


## まとめ

分類は十分に一般的な問題であるため、独自の便利な機能が保証されます。分類において最も重要なのは、分類器の*精度*です。私たちは主に精度を重視することが多いですが、統計的および計算上の理由から、他のさまざまな目的を最適化するために分類器をトレーニングしていることに注意してください。ただし、トレーニング中にどの損失関数が最小化されたかに関係なく、分類器の精度を経験的に評価するための便利な方法があると便利です。

## 演習
1. 検証損失を $L_v$ で表し、$L_v^q$ をこのセクションの損失関数の平均化によって計算されたクイック アンド ダーティ推定値とします。最後に、最後のミニバッチの損失を $l_v^b$ で示します。 $L_v$ を $L_v^q$、$l_v^b$、およびサンプルとミニバッチのサイズで表現します。
1. 簡単で汚い推定値 $L_v^q$ に偏りがないことを示します。つまり、$E[L_v] = E[L_v^q]$ であることを示します。それでも代わりに $L_v$ を使用したいのはなぜですか?
1.  $y$ を見たときに $y&#39;$ を推定するペナルティを $l(y,y&#39;)$ で表すマルチクラス分類損失があり、確率 $p(y \mid x)$ が与えられた場合、次のルールを定式化します。 $y&#39;$ の最適な選択。ヒント: $l$ と $p(y \mid x)$ を使用して、予想される損失を表します。



[ディスカッション](https://discuss.d2l.ai/t/6809)
